# Image Sort by Saturation
---
This package is designed to run on Google Colab and will sort Google Drive photos into color and grayscale categories.
- Refer to the [README.md on GitHub](https://github.com/jamesswartwood/image-sort-by-saturation/blob/main/README.md) for usage instructions
---

## Configure Program Settings
Update the values below to allow this tool to function as intended on your system. This block of code functions as the program settings.

In [1]:
FOLDER_PATH = ""     # Path in your Google Drive to target folder (Ex: "this/is/the/target/folder")
THRESHOLD = 0.02     # Decrease threshold for more sensitivity to color

## Mount Google Drive
These blocks of code let you mount your Google Drive and then navigate to the desired folder.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd {'gdrive/My Drive/' + FOLDER_PATH}

## Package Imports
This code block imports necessary Python packages.

In [4]:
import os
import glob #for loading images from a directory
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import random
import numpy as np

## Function Definitions
These blocks of code define the functions that will be used in the program execution.

In [5]:
# Establish the destination directories for sorted images
def establish_directories():
    %mkdir 'color_images'
    %mkdir 'grayscale_images'

In [6]:
# Returns image fitted to the desired size
def standardize(image):
    return cv2.resize(image, (1100, 600))

In [7]:
# Find the average saturation of an image
def avg_saturation(rgb_image):
    # Convert image to HSV
    hsv = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2HSV)

    # Add up all the pixel values in the V channel
    sum_brightness = np.sum(hsv[:,2,:])
    area = 600*1100.0  # pixels
    
    # find the avg
    avg = sum_brightness/area
    
    return avg

In [8]:
# This function should take in RGB image input
def estimate_label(rgb_image, threshold):
    
    # Extract average brightness feature from an RGB image 
    avg = avg_saturation(rgb_image)
        
    # Use the avg brightness feature to predict a label (0, 1)
    predicted_label = 0

    if(avg > threshold):
        # if the average saturation is above the threshold value, we classify it as "color"
        predicted_label = 1
    # else, the pred-cted_label can stay 0 (it is predicted to be "grayscale")
    
    return predicted_label

In [9]:
# Runs the image sort program and tracks progess
def run_image_sort():
    color_count = 0
    grayscale_count = 0

    establish_directories()

    for file in glob.glob(os.path.join("*")):
        try:
            # Read in the image
            image = mpimg.imread(file)

            # Check if the image exists/if it's been correctly read-in
            if not image is None:

                # Updates image size
                image = standardize(image)

                # Predicts average saturation and sorts
                if (estimate_label(image, THRESHOLD)):
                    %mv {"\'" + str(file) + "\'"} {"\'color_images/" + str(file) + "\'"}
                    color_count += 1
                else:
                    %mv {"\'" + str(file) + "\'"} {"\'grayscale_images/" + str(file) + "\'"}
                    grayscale_count += 1

        except:
            continue
    
    print("Number of color images sorted:", color_count)
    print("Number of grayscale images sorted:", grayscale_count)

## Double-checking Settings
These code blocks will allow you to view data that might indicate errors in your settings. If all appears well, proceed to the program execution.

In [ ]:
print("Google Drive target directory:", os.getcwd())

num_items = 0
for file in glob.glob(os.path.join("*")):
    num_items += 1
print("\nNumber of items in target directory:", num_items)

In [ ]:
print("\nItems list:")
%ls

## Program Execution
This one line of code will run the program.

In [ ]:
run_image_sort()